In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import sys 
sys.path.append('/content/drive/MyDrive/ECE_661/661_Final_Project')

In [ ]:
# import necessary dependencies
import argparse
import os
import time
import tools
import datetime
from tqdm import tqdm_notebook as tqdm
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F

import numpy as np

In [ ]:
# define the ResBlock to be used in ResNet;
class ResBlock(nn.Module):
    def __init__(self, input_dim, output_dim, k_size=3, stride_bool=False):
        super(ResBlock, self).__init__()
        if(stride_bool == True):
          self.conv1 = nn.Conv2d(input_dim, output_dim, kernel_size=k_size, stride=2, padding=1)
          self.resconv = nn.Conv2d(input_dim, output_dim, kernel_size=1, stride=2)
        else:
          self.conv1 = nn.Conv2d(input_dim, output_dim, kernel_size=k_size, padding=1)
          self.resconv = nn.Conv2d(input_dim, output_dim, kernel_size=1)

        if input_dim != output_dim:
          input_dim = output_dim
        self.bn1 = nn.BatchNorm2d(output_dim)
        self.conv2 = nn.Conv2d(input_dim, output_dim, kernel_size=k_size, padding=1)
        self.bn2 = nn.BatchNorm2d(output_dim)

    def forward(self, x):
        y = x
      # Note shortcut will just be x if of same shape as out
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out)) 
        if (x.shape == out.shape):
          out = out + y
          out = F.relu(out)
          return out
        else:
          y = self.resconv(y)
          out = out.add(y)
          out = F.relu(out)
          return out

In [ ]:
class ResNet(nn.Module):
  def __init__(self):
    super(ResNet, self).__init__()
    self.conv1 = nn.Conv2d(3, 16, 3, padding=1)
    self.bn1 = nn.BatchNorm2d(16)
    # 16 filter res blockss
    self.res1 = ResBlock(16, 16)
    self.res2 = ResBlock(16, 16)
    self.res3 = ResBlock(16, 16)
    # Now 32 filter res blocks
    self.res4 = ResBlock(16, 32, stride_bool=True)
    self.res5 = ResBlock(32, 32)
    self.res6 = ResBlock(32, 32)
    # Now 64 filter res blocks 
    self.res7 = ResBlock(32, 64, stride_bool=True)
    self.res8 = ResBlock(64, 64)
    self.res9 = ResBlock(64, 64)

    self.fc = nn.Linear(64, 10)
    self.fc2 = nn.Linear(64,4)

  def forward(self, x):
    out = F.relu(self.bn1(self.conv1(x)))

    # ReLU contained within forward method of res blocks
    
    out = self.res1(out)
    out = self.res2(out)
    out = self.res3(out)

    out = self.res4(out)
    out = self.res5(out)
    out = self.res6(out)

    out = self.res7(out)
    out = self.res8(out)
    out = self.res9(out)

    out = F.adaptive_avg_pool2d(out, 1)
    out = out.view(out.size(0), -1)
    outt = self.fc(out)
    outr = self.fc2(out)
    # out = F.softmax(out)
    return outt, outr

In [ ]:
class Resnet(nn.Module):
    def __init__(self):
        super(Resnet, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, 3, stride = 1, padding = 1) # I am not sure about this output, should it be 32 or 3? Also coudln't I just use identities by forcing 
        self.conv1_bn = nn.BatchNorm2d(16)

        #starting first section of 6 - feature map 32, channel (features) 16
        self.conv2 = nn.Conv2d(16,16,3, stride = 1, padding = 1)
        self.conv2_bn = nn.BatchNorm2d(16)
        self.conv3 = nn.Conv2d(16,16,3, padding = 1)
        self.conv3_bn = nn.BatchNorm2d(16)
        self.conv4 = nn.Conv2d(16,16,3, padding = 1)
        self.conv4_bn = nn.BatchNorm2d(16)
        self.conv5 = nn.Conv2d(16,16,3, padding = 1)
        self.conv5_bn = nn.BatchNorm2d(16)
        self.conv6 = nn.Conv2d(16,16,3, padding = 1)
        self.conv6_bn = nn.BatchNorm2d(16)
        self.conv7 = nn.Conv2d(16,16,3, padding = 1) #it would be here that I need a 1x1 convolution on my bypass that gets fed to lower stuff 
        self.conv7_bn = nn.BatchNorm2d(16)
        self.bp4_conv = nn.Conv2d(16,32,1, stride = 2)
        self.bp4_bn = nn.BatchNorm2d(32) 

        

        #Now starting second section of 6 - now feature map 16, channel (features) 32
        self.conv8 = nn.Conv2d(16,32,3, stride = 2, padding = 1) # HERE COULD BE AN ISSUE - I AM NOT SURE IF I SHOULD PAD OR NOT 
        self.conv8_bn = nn.BatchNorm2d(32) 
        self.conv9 = nn.Conv2d(32,32,3, padding = 1)
        self.conv9_bn = nn.BatchNorm2d(32) 
        self.conv10 = nn.Conv2d(32,32,3, padding = 1)
        self.conv10_bn = nn.BatchNorm2d(32) 
        self.conv11 = nn.Conv2d(32,32,3, padding = 1)
        self.conv11_bn = nn.BatchNorm2d(32) 
        self.conv12 = nn.Conv2d(32,32,3, padding = 1)
        self.conv12_bn = nn.BatchNorm2d(32) 
        self.conv13 = nn.Conv2d(32,32,3, padding = 1) # need 1x1 conv here on bypass that gets fed to lower stuff 
        self.conv13_bn = nn.BatchNorm2d(32)
        self.bp7_conv = nn.Conv2d(32,64,1, stride = 2)
        self.bp7_bn = nn.BatchNorm2d(64) 

        # now starting third section of 6 - feature map is 8, channel (features) 64 (8x8x64)
        self.conv14 = nn.Conv2d(32,64,3, stride = 2, padding = 1) # SAME THING MAYBE I NEED TO PADD MAYBE I DON'T NEED TO 
        self.conv14_bn = nn.BatchNorm2d(64) 
        self.conv15 = nn.Conv2d(64,64,3, stride = 1, padding = 1)
        self.conv15_bn = nn.BatchNorm2d(64) 
        self.conv16 = nn.Conv2d(64,64,3, stride = 1, padding = 1)
        self.conv16_bn = nn.BatchNorm2d(64) 
        self.conv17 = nn.Conv2d(64,64,3, stride = 1, padding = 1)
        self.conv17_bn = nn.BatchNorm2d(64) 
        self.conv18 = nn.Conv2d(64,64,3, stride = 1, padding = 1)
        self.conv18_bn = nn.BatchNorm2d(64) 
        self.conv19 = nn.Conv2d(64,64,3, stride = 1, padding = 1)
        self.conv19_bn = nn.BatchNorm2d(64)  
       

        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.fc1   = nn.Linear(64,10)
        self.fc2 = nn.Linear(64,4) 

    # THIS IS WHERE I need to fit it all together and worry about my residual stuff 
    def forward(self, x):
        out = F.relu(self.conv1_bn(self.conv1(x))) # this is okay, goes after conv and before activation functionp

        bp1 = out # SAVE THE BYPASS DATA HERE, MAYBE COPY? Maybe not 
      
        out = F.relu(self.conv2_bn(self.conv2(out)))

        out = self.conv3_bn(self.conv3(out))
  
        # IN here I need to add the first bypass data to the current out, then I feed it to the next guy - need to think a little about when I need to do the convolution on the BP
        out = bp1 + out #This totally works! Very chill and cool of pytorch lol
        out = F.relu(out)
        bp2 = out # maybe copy, maybe not, we'll see 
        out  = F.relu(self.conv4_bn(self.conv4(out)))
        out = self.conv5_bn(self.conv5(out))
        out = bp2 + out
        out = F.relu(out)
        bp3 = out
        out = F.relu(self.conv6_bn(self.conv6(out)))
        out = self.conv7_bn(self.conv7(out))
        out = out + bp3
        out = F.relu(out)
        bp4 = out
        bp4_ready = self.bp4_bn(self.bp4_conv(bp4))
        out = F.relu(self.conv8_bn(self.conv8(out)))
        out = self.conv9_bn(self.conv9(out))
        out = bp4_ready+out
        out = F.relu(out)
        bp5 = out
        out = F.relu(self.conv10_bn(self.conv10(out)))
        out = self.conv11_bn(self.conv11(out))
        out = out + bp5
        out = F.relu(out)
        bp6 = out
        out = F.relu(self.conv12_bn(self.conv12(out)))
        out = self.conv13_bn(self.conv13(out))
        out = out + bp6
        out = F.relu(out)
        bp7 = out
        bp7_ready = self.bp7_bn(self.bp7_conv(out))
        out = F.relu(self.conv14_bn(self.conv14(out)))
        out = self.conv15_bn(self.conv15(out))
        out = bp7_ready+out
        out = F.relu(out)
        bp8 = out
        out = F.relu(self.conv16_bn(self.conv16(out)))
        out = self.conv17_bn(self.conv17(out))
        out = out + bp8
        out = F.relu(out)
        bp9 = out
        out = F.relu(self.conv18_bn(self.conv18(out)))
        out = self.conv19_bn(self.conv19(out))
        out = out + bp9
        out = F.relu(out)
        out = self.avg_pool(out)


        out = out.view(out.size(0), -1) # IF THIS BREAKS THEN CHECK HERE, Might need it might not
        #avg pooling here
        #fully connected here
        outt = self.fc1(out)
        outr = self.fc2(out)
        return outt, outr

In [ ]:
# do NOT change these
#import tools
from tools.dataset import CIFAR10
from torch.utils.data import DataLoader

# a few arguments, do NOT change these
DATA_ROOT = "./data"
TRAIN_BATCH_SIZE = 256
VAL_BATCH_SIZE = 100

#############################################
# your code here
# construct dataset
test_set = CIFAR10(
    root=DATA_ROOT, 
    mode='test', 
    download=True,
)


test_loader = DataLoader(
    test_set, 
    batch_size=256,  # your code
    shuffle=False,     # your code
    num_workers=4
)
#############################################

0it [00:00, ?it/s]

Extracting ./data/cifar10_test_F22.zip to ./data
Files already downloaded and verified


/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py:554: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [ ]:
test_labels = np.asarray(np.load('/content/drive/MyDrive/ECE_661/661_Final_Project/tools/test-labels-batch.npy'))

In [ ]:
import shutil
from PIL import Image
from torch.utils.data import Dataset as VisionDataset
from tools.utils import check_integrity, download_and_extract_archive
import torchvision
import torchvision.transforms as transforms
#############################################
# your code here
# construct dataset
transform_val = transforms.Compose([transforms.ToTensor(), transforms.Normalize(mean=(0.4914, 0.4822, 0.4465), std=(0.2023, 0.1994, 0.2010))])

test_set = CIFAR10(
    root=DATA_ROOT, 
    mode='test', 
    download=True,
    transform=transform_val
)


test_loader = DataLoader(
    test_set, 
    batch_size=256,  # your code
    shuffle=False,     # your code
    num_workers=4
)
#############################################

Using downloaded and verified file: ./data/cifar10_test_F22.zip
Extracting ./data/cifar10_test_F22.zip to ./data
Files already downloaded and verified


/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py:554: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
if device =='cuda':
    print("Run on GPU...")
else:
    print("Run on CPU...")
  
net = ResNet().to(device)
state_dict = torch.load("/content/drive/MyDrive/ECE_661/661_Final_Project/resnet_base.pth")
net.load_state_dict(state_dict["state_dict"])

Run on GPU...


<All keys matched successfully>

In [ ]:
def test_net(net, dataloader, batch_size):
  correct = 0
  total = 10000
  size_total = 0
  i = 0
  for x in dataloader:
    x = x.to(device)
    out = net(x)[0]
    size_total += out.size()[0]
    targets = torch.Tensor(test_labels[i:i+batch_size]).to(device)
    i += batch_size

    _, predicted = torch.max(out, 1)
    correct += predicted.eq(targets).sum()
  # print(size_total)
  # print(correct)
  return correct / total

In [ ]:
net = ResNet().to(device)
state_dict = torch.load("/content/drive/MyDrive/ECE_661/661_Final_Project/resnet_base.pth")
net.load_state_dict(state_dict["state_dict"])
base = test_net(net, test_loader, 256)

In [ ]:
lam = [0.15, 0.3, 0.5, 0.7, 0.85]
hole_accs = []
for hole in lam:
  net = ResNet().to(device)
  state_dict = torch.load("/content/drive/MyDrive/ECE_661/661_Final_Project/resnet_mixup_hyp{}.pth".format(hole))
  net.load_state_dict(state_dict["state_dict"])
  out = test_net(net, test_loader, 256)
  hole_accs.append(out)
  print(f"Accuracy for {hole} = {out}")

Accuracy for 0.15 = 0.8306999802589417
Accuracy for 0.3 = 0.8623999953269958
Accuracy for 0.5 = 0.8317999839782715
Accuracy for 0.7 = 0.85999995470047
Accuracy for 0.85 = 0.8259999752044678


In [ ]:
lengths = [2, 4, 8, 16]
lengths_accs = []
for length in lengths:
  net = ResNet().to(device)
  state_dict = torch.load("/content/drive/MyDrive/ECE_661/661_Final_Project/resnet_cutout_only_finetune_length{}.pth".format(length))
  net.load_state_dict(state_dict["state_dict"])
  out = test_net(net, test_loader, 256)
  lengths_accs.append(out)
  print(f"Accuracy for {length} = {out}")

Accuracy for 2 = 0.8125999569892883
Accuracy for 4 = 0.8176999688148499
Accuracy for 8 = 0.824999988079071
Accuracy for 16 = 0.8306999802589417


In [ ]:
net = ResNet().to(device)
state_dict = torch.load("/content/drive/MyDrive/ECE_661/661_Final_Project/resnet_cutout.pth")
net.load_state_dict(state_dict["state_dict"])
cutout = test_net(net, test_loader, 256)

In [ ]:
net = ResNet().to(device)
state_dict = torch.load("/content/drive/MyDrive/ECE_661/661_Final_Project/resnet_mixup.pth")
net.load_state_dict(state_dict["state_dict"])
mixup = test_net(net, test_loader, 256)

In [ ]:
net = ResNet().to(device)
state_dict = torch.load("/content/drive/MyDrive/ECE_661/661_Final_Project/resnet_base_noreg_50.pth")
net.load_state_dict(state_dict["state_dict"])
test_net(net, test_loader, 256)

tensor(0.8144, device='cuda:0')

In [ ]:
net = ResNet().to(device)
state_dict = torch.load("/content/drive/MyDrive/ECE_661/661_Final_Project/resnet_mixup_noreg_50.pth")
net.load_state_dict(state_dict["state_dict"])
test_net(net, test_loader, 256)

tensor(0.8347, device='cuda:0')

In [ ]:
net = ResNet().to(device)
state_dict = torch.load("/content/drive/MyDrive/ECE_661/661_Final_Project/resnet_cutout_50_noreg.pth")
net.load_state_dict(state_dict["state_dict"])
test_net(net, test_loader, 256)

tensor(0.8300, device='cuda:0')

In [ ]:
net = ResNet().to(device)
state_dict = torch.load("/content/drive/MyDrive/ECE_661/661_Final_Project/resnet_mixup_cutout.pth")
net.load_state_dict(state_dict["state_dict"])
mixup_cutout = test_net(net, test_loader, 256)

In [ ]:
net = ResNet().to(device)
state_dict = torch.load("/content/drive/MyDrive/ECE_661/661_Final_Project/resnet_aux_cutout.pth")
net.load_state_dict(state_dict["state_dict"])
aux_cutout = test_net(net, test_loader, 256)

In [ ]:
# print(f"Base: {base}")
# print(f"Mixup: {mixup}")
# print(f"Cutout: {cutout}")
# print(f"Auxiliary Rotation: {base}")
# print(f"Mixup + Cutout: {mixup_cutout}")
print(f"Aux + Cutout: {aux_cutout}")

Aux + Cutout: 0.8694999814033508
